# Importation des modules

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from ast import literal_eval
import csv
import re

In [2]:
RefNom = pd.read_csv("./../Referentiels/ReferentielNoms.csv",converters={"FormesNonNormalisées": literal_eval}).set_index("NomNormalisé")
RefPrenom = pd.read_csv("./../Referentiels/ReferentielPrenoms.csv", converters={"FormesNonNormalisées": literal_eval}).set_index("PrénomNormalisé")

In [3]:
def Parsing(CheminRelatifFichier, DivCC, NomFichier):
    soup = BeautifulSoup(open(CheminRelatifFichier, 'r'))
    
    #On commence par itérer à travers les balises div typées 'date'.
    
    for DivDate in soup.find_all('div', attrs={'type' : 'date'}):
        
        #On crée une variable pour générer un identifiant temporaire.
        n = 0
        
        #On crée une variable où l'on enregistre la valeur de la date.
        Année = DivDate['n']
        
        
        #On crée notre Liste qui correspondra au conseil et qu'on transformera plus tard en DataFrame.
        
        ListeDicosPersonnes = []
        
        #On itère à travers les personnes.
        
        for persname in DivDate.find_all('persname'):
            
            
            DicoPersonne = {}
            
            #On crée l'identifiant temporaire pour chaque individu
            n = n + 1
            length = len(str(n))
            nID = str(n).zfill(4)               
            DicoPersonne['id'] = NomFichier + '_' + str(Année) + '_' + nID
            
            
            #On récupère les informations présentes dans les balises parentes.
            
            for div in persname.find_parents('div'):
                
                #Le corps civique.
                if div['type'] == 'ordre' :
                    
                    DicoPersonne['corps civique'] = div['n']
                
                #La corporation.
                if div['type'] == 'corporation':
                    
                    DicoPersonne['corporation'] = div['n']
                
                #La magistrature
                if div['type'] == 'magistrature':
                    
                    DicoPersonne['magistrature'] = div['n']
                    DicoPersonne['anneeMagistrature'] = Année
                
                #Le poêle
                if div['type'] == 'poele':
                    
                    DicoPersonne['poele'] = div['n']
            
            #On récupère l'occurence
            occurence = persname.get_text()
            occurence = re.sub(r"\n", " ", occurence)
            DicoPersonne['occurence'] = occurence
            
            #On normalise le nom de famille
            for surname in persname.find_all('surname'):
                DataToComp = surname.get_text()
                DataToComp = re.sub(r"\n", " ", DataToComp)
                DataToComp = DataToComp.lower()
                if type(DataToComp) == float:
                    DicoPersonne['nom'] = ''
                for j in range (0, len(RefNom.index)):
                    CheckN = RefNom.iloc[j,1]
                    for element in CheckN:
                        if DataToComp == element:
                            DicoPersonne['nom'] = RefNom.index[j]
                            
            #Puis le surnom                
            for addName in persname.find_all('addname'):
                DataToComp = addName.get_text()
                DataToComp = re.sub(r"\n", " ", DataToComp)
                DataToComp = DataToComp.lower()
                if type(DataToComp) == float:
                    DicoPersonne['surnom'] = ''
                for j in range (0, len(RefNom.index)):
                    CheckN = RefNom.iloc[j,1]
                    for element in CheckN:
                        if DataToComp == element:
                            DicoPersonne['surnom'] = RefNom.index[j]
            for genname in persname.find_all('genname'):
                DataToComp = genname.get_text()
                DataToComp = re.sub(r"\n", " ", DataToComp)
                DataToComp = DataToComp.lower()
                if type(DataToComp) == float:
                    DicoPersonne['genName'] = ''
                for j in range (0, len(RefNom.index)):
                    CheckN = RefNom.iloc[j,1]
                    for element in CheckN:
                        if DataToComp == element:
                            DicoPersonne['genName'] = RefNom.index[j]
            
            #On normalise le prénom
            for forename in persname.find_all('forename'):
        
                # Loop pour prénom normalisé.
                DataToComp = forename.get_text()
                DataToComp = re.sub(r"\n", " ", DataToComp)
                DataToComp = DataToComp.lower()
                if type(DataToComp) == float:
                    DicoPersonne['prenom'] = ''
                for j in range (0, len(RefPrenom.index)):
                    CheckN = RefPrenom.iloc[j,2]
                    for element in CheckN:
                        if DataToComp == element:
                            DicoPersonne['prenom'] = RefPrenom.index[j]
            
            #On récupère également les différents titres portés par la personne ainsi que ses rôles.
            Roles = []
            for rolename in persname.find_all('rolename'):
                Role = rolename.get_text()
                Roles.append(Role)
                DicoPersonne['titres_roles'] = Roles
                
            #Finalement on récupère les métadonnées liées à chaque mention de personne
            page = persname.find_previous('pb')
            DicoPersonne['page'] = page['n']
            
            column = persname.find_previous('cb')
            DicoPersonne['colonne'] = column['n']
        
            line = persname.find_previous('lb')
            DicoPersonne['ligne'] = line['n']
            
            ListeDicosPersonnes.append(DicoPersonne)
            
            DfListe = pd.DataFrame(ListeDicosPersonnes)
            DfListe = DfListe.reindex(columns=['id', 'anneeMagistrature', 'magistrature', 'occurence', 'prenom', 'nom', 'surnom', 'genName', 'titres_roles', 'corps civique', 'corporation', 'poele', 'page', 'colonne', 'ligne'])
            DfListe.fillna('NaN')
            DfListe.index += 1
            DfListe.to_csv("./../Sources/ListesConseil/" + Année + "ListeConseil.csv")        
        

In [4]:
Parsing('./../Sources/EditionsXML/aves_4R_p131-220.xml', 'TEI', 'aves_aa_4R')